In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn import metrics
import os
import matplotlib.pyplot as plt
import PIL.Image

In [ ]:
model = keras.models.load_model('region_classifier_1.h5')

In [ ]:
class_names = ['adequate', 'blood', 'clot']

folder_path = "all_region_val"
img_height = 1000
img_width = 1000

img_list = os.listdir(folder_path)


y_true = []
for img_idx in range(len(img_list)):
    img_filename = img_list[img_idx]
    if("adequate" in img_filename):
        y_true.append([1,0,0])
    elif("blood" in img_filename):
        y_true.append([0,1,0])
    else:
        y_true.append([0,0,1])

print(y_true)
print(len(y_true))

In [ ]:
y_pred = []

for cur_img in img_list:
    img_path = os.path.join(folder_path, cur_img)
    img = tf.keras.utils.load_img(img_path, target_size = (img_height, img_width))

    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    conf_values = list(score.numpy())
    y_pred.append(conf_values)

print(y_pred)

In [ ]:
class_names = ["Adequate", "Blood", "Clot"]
y_true_arr = np.array(y_true)
y_pred_arr = np.array(y_pred)

fpr_list, tpr_list, auc_scores = [], [], []
n_classes = y_true_arr.shape[1]

for i in range(n_classes):
    fpr, tpr, _ = metrics.roc_curve(y_true_arr[:, i], y_pred_arr[:, i])
    # print(f"{i}:", fpr, tpr)
    if i==2:
        for j in range(len(fpr)):
            print(f"({fpr[j]}, {tpr[j]})", end=", ")
    auc = metrics.roc_auc_score(y_true_arr[:, i], y_pred_arr[:, i])
    fpr_list.append(fpr)
    tpr_list.append(tpr)
    auc_scores.append(auc)

plt.figure(figsize=(10, 8))
# for i in range(n_classes):
#     plt.plot(fpr_list[i], tpr_list[i], label=f"Class {class_names[i]} (AUC = {auc_scores[i]:.2f})")
for i in range(n_classes):
    plt.plot(fpr_list[i], tpr_list[i], label=f"{class_names[i]}")


# Plot formatting
plt.plot([0, 1], [0, 1], "k--")  # Diagonal line
plt.title("ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.show()